In [50]:
import requests
import json
import bs4
import time
import numpy as np
import pandas as pd

import sys
import signal

from itertools import islice   


import CONFIG

In [2]:
def getBSFromURL(url):
    try:
        r = requests.get(url, headers = CONFIG.HEADER)
        return getBS(r.text)
    
    except requests.exceptions.RequestException as e:
        print("Connection Error")

def getBS(html):
    html_bs = bs4.BeautifulSoup(html, 'html.parser')
    return html_bs

In [32]:
# a very simple sleeping function to avoid getting banned
def ImNotRobot(t = 5):
    sleepTime = np.random.uniform(t,t+5)
    print(f"sleeping {sleepTime} seconds")
    time.sleep(sleepTime)
    
    
def save():
    all_issues_df.to_csv(all_issues_file_name, index = False)
    
def saveAndQuit(sig=None, frame=None):
    save()
    sys.exit(0)


## Handle the scraping first

To avoid error occuring during the parsing, I scrape all the html containing the article information first.
Abstracts and author lists are packed in the webpage of an issue. Extraction of data will be handled afterweard once the raw html is downloaded.

In [55]:
all_articles_file_name = f"{CONFIG.JOURNAL_NAME}.csv"
all_issues_file_name = f"{CONFIG.JOURNAL_NAME}_all_issue.csv"

with open(all_issues_file_name) as f:
    all_issues_df = pd.read_csv(f)
needed_issues = all_issues_df[all_issues_df['scraped'] != 1]
display(needed_issues)
num = len(needed_issues)
counter = 1

for index, issue in needed_issues.iterrows():
    url = CONFIG.DOMAIN_URL + issue['URL']
    print(f"Scraping {counter} / {num}")
    try:
        issue_bs = getBSFromURL(url)
        issue_main_bs = issue_bs.select('.table-of-content__section-body')[0]
        all_issues_df.loc[index, 'bs'] = str(issue_main_bs)
        all_issues_df.loc[index, 'scraped'] = 1
    except KeyboardInterrupt as e:
        print("Terminated.")
        sys.exit(1)
    except Exception as e:
        print(e)
        all_issues_df.loc[index, 'scraped'] = -1
        continue
    finally:
        all_issues_df.to_csv(all_issues_file_name, index = False)
        counter += 1
        

,URL,Volume,Number,Date,scraped,bs


## Parsing the Data

Now with all the raw data scraped, we are ready to do the parsing. We iterate through the issues
1. get the Volume , Number, Month, Year for the issue
2. turn the bs string into soup
3. select the ones with author
4. get their parents
5. get author elements
6. try get abstract. Return empty is none

In [117]:
def parseVolume(v):
    v = v.strip()
    return v.split('Volume ')[1]

def parseNumber(n):
    n = n.strip()
    return n.split('Number ')[1]

def parseDate(d):
    d = d.strip()
    
    ## split by comma first
    split_comma = d.split(',')
    
    ## like Oct., 1996
    if len(split_comma) == 2:
        month = split_comma[0]
        year = split_comma[1]
        
        return month.strip(), year.strip()
    
    ## else is like October 1998
    split_space = d.split(' ')
    if len(split_space) != 2:
        return '', ''
    
    month = split_space[0]
    year = split_space[1]
    
    return month, year

def getArticlesWithAuthor(s):
    c_articels = s.select('.issue-item__loa')
    if not c_articels:
        return None
    return [a.parent for a in c_articels]

def getArticleName(s):
    c_issue_name = s.select('.issue-item__title a')
    if not c_issue_name :
        return ''
    return c_issue_name[0].text

def getArticleURL(s):
    c_issue_url = s.select('.issue-item__title a')
    if not c_issue_url :
        return ''
    return CONFIG.DOMAIN_URL + c_issue_url[0]['href']


def getAbstract(s):
    c_abstract = s.select('.accordion__content p')
    if not c_abstract:
        return ''
    return c_abstract[0].text

def getAuthors(s):
    c_authors = s.select('.hlFld-ContribAuthor')
    if not c_authors:
        return []
    return [" ".join(a.contents[0].split()) for a in c_authors]

In [127]:
with open(all_issues_file_name) as f:
    all_issues_df = pd.read_csv(f)

all_article = []
for index, issue in all_issues_df.iterrows():
    print(index)
    volume = parseVolume(issue['Volume'])
    number = parseNumber(issue['Number'])
    month, year = parseDate(issue['Date'])
    
    issue_bs = getBS(issue['bs'])
    articles = getArticlesWithAuthor(issue_bs)
    for article in articles:
        title = getArticleName(article)
        url = getArticleURL(article)
        abstract = getAbstract(article)
        authors = getAuthors(article)
        
        to_append = [
            url, title, volume, number, month, year, abstract
        ] + authors
        all_article.append(to_append)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [128]:
col_num = max([len(a) for a in all_article])

colName = [
    'Source URL',
    'Title',
    'Issue Vol.',
    'Issue No.',
    'Month',
    'Year',
    'Abstract'
] + [f'Author {i+1}' for i in range(col_num - 7)]

all_article_df = pd.DataFrame(all_article)

In [129]:
all_article_df.to_csv(f'{CONFIG.JOURNAL_NAME}.csv', header=colName, index=False)

In [100]:
bst = getBS(all_issues_df.loc[300,'bs'])

In [107]:
getAuthors(getArticlesWithAuthor(bst)[2])

['Rosemary Avery', 'Donald Kenkel', 'Dean R. Lillard', 'Alan Mathios']